<a href="https://colab.research.google.com/github/sakurasakura1996/Pytorch-start-learning/blob/master/Dive_into_DL_Pytorch_4_5_%E8%AF%BB%E5%8F%96%E5%92%8C%E5%AD%98%E5%82%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4.5读取和存储
使用save load 分别存储和读取Tensor:
    save 使用Python的pickle实用程序将对象进行序列化，然后将序列化的对象保存到disk中，使用save可以保存各种对象，包括模型、张量和字典等
    load使用pickle unpickle工具将pickle的对象文件反序列化为内存


In [0]:
import torch
from torch import nn

x = torch.ones(3)
torch.save(x, 'x.pt')

In [2]:
# 然后再从数据中将其读入内存
x2 = torch.load('x.pt')
print(x2)

tensor([1., 1., 1.])


In [3]:
# 还可以存储一个Tensor列表并读回内存
y = torch.zeros(4)
torch.save([x,y],'xy.pt')

xy_list_load = torch.load('xy.pt')
print(type(xy_list_load))
print(len(xy_list_load))   # 一个tensor视为一个列表元素
print(xy_list_load)

<class 'list'>
2
[tensor([1., 1., 1.]), tensor([0., 0., 0., 0.])]


In [4]:
# 当然 存储并读取 从一个字符串映射到 Tensor的字典
torch.save({
    'x':x,
    'y':y
},'xy_dict.pt')

xy_dict_load = torch.load('xy_dict.pt')
for key, value in xy_dict_load.items():
    print(key,value)

x tensor([1., 1., 1.])
y tensor([0., 0., 0., 0.])


In [5]:
# 读写模型  state_dict
# 在pytorch中，Module的可学习参数（即权重和偏差），模块模型包含在参数中（通过model.parameters()访问）。state_dict是一个从参数名称映射到参数Tensor的字典对象
class MLP(nn.Module):
    def __init__(self):
        super(MLP,self).__init__()
        self.hidden = nn.Linear(3, 2)   # 注意这里没有逗号啊，加了逗号发现后面state_dict()没有隐藏层的参数了
        self.act = nn.ReLU()
        self.output = nn.Linear(2, 1)
    
    def forward(self, x):
        a = self.act(self.hidden(x))
        return self.output(a)
    
net = MLP()
print(net.state_dict())   # OrderedDict类型
print(net.parameters())  # 打印出来都是一个 generator 迭代器
print(net.named_parameters())

print('-----------------------')
for param in net.parameters():
    print(param)
    print(param.data)
    
print('-----------------------')
for name, param in net.named_parameters():
    print(name, param.data)
    

OrderedDict([('hidden.weight', tensor([[ 0.0757, -0.0958, -0.1102],
        [ 0.4308,  0.2259,  0.5228]])), ('hidden.bias', tensor([ 0.3304, -0.4504])), ('output.weight', tensor([[ 0.3593, -0.5808]])), ('output.bias', tensor([0.6307]))])
<generator object Module.parameters at 0x7fbea4069db0>
<generator object Module.named_parameters at 0x7fbea4069db0>
-----------------------
Parameter containing:
tensor([[ 0.0757, -0.0958, -0.1102],
        [ 0.4308,  0.2259,  0.5228]], requires_grad=True)
tensor([[ 0.0757, -0.0958, -0.1102],
        [ 0.4308,  0.2259,  0.5228]])
Parameter containing:
tensor([ 0.3304, -0.4504], requires_grad=True)
tensor([ 0.3304, -0.4504])
Parameter containing:
tensor([[ 0.3593, -0.5808]], requires_grad=True)
tensor([[ 0.3593, -0.5808]])
Parameter containing:
tensor([0.6307], requires_grad=True)
tensor([0.6307])
-----------------------
hidden.weight tensor([[ 0.0757, -0.0958, -0.1102],
        [ 0.4308,  0.2259,  0.5228]])
hidden.bias tensor([ 0.3304, -0.4504])
output

In [6]:
# 注意，只有具有可学习参数的层（卷积层、线性层等）才有state_dict中的条目。优化器optim 也有一个state_dict,其中包含关于优化器状态以及所使用的超参数的信息
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer.state_dict()

{'param_groups': [{'dampening': 0,
   'lr': 0.001,
   'momentum': 0.9,
   'nesterov': False,
   'params': [140455476532280,
    140455476532208,
    140455476532352,
    140455476532424],
   'weight_decay': 0}],
 'state': {}}

保存和加载模型  pytorch中保存和加载训练模型有两种常见的方法：
1.仅保存和加载模型参数（state_dict);   
2.保存和加载整个模型
    1.torch.save(model.state_dict(), PATH)   建议文件后缀名是 pt 或者 pth
      加载： model = TheModelClass(*args, **kwargs)   model.load_state_dict(torch.load(PATH))
    2.torch.save(model,PATH)
      加载： model = torch.load(PATH)

# 4.6GPU计算
以上都还是再用cpu再跑程序，这一节介绍用 GPU来计算。由于我这笔记本垃圾的不行，看来这一节的学习就搬到 google colab上去学习啦，不过还是在这里写一遍吧，书上没太多内容

In [11]:
!nvidia-smi
# 查看显卡信息，这里是 Tesla P100  16GB显存，卧槽还可以啊

Tue Mar  3 13:30:48 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    29W / 250W |     10MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [12]:
# pytorch可以指定用来存储和计算的设备，如使用内存的CPU  或者使用显存的GPU。默认情况下，pytorch会将数据创建再内存，然后用CPU计算
# 用torch.cuda.is_available()
print(torch.cuda.is_available())
print(torch.cuda.device_count())  # 查看GPU数量
print(torch.cuda.current_device())   # 查看当前GPU 索引号，索引号从0开始，由于这里没有所有会报错啦  呜呜呜
print(torch.cuda.get_device_name())

True
1
0
Tesla P100-PCIE-16GB


In [13]:
# 默认tensor会被存在内存上。因此之前打印tensor信息时看不到GPU相关标识
x =torch.tensor([1,2,3])
print(x)

tensor([1, 2, 3])


In [15]:
# 使用 .cuda() 可以将 CPU 上的tensor转换复制到 GPU上，如果有多快卡，可以通过cuda(i)来表示第i块GPU及相应的显存 （i从0开始） 且cuda(0) 和 cuda()相同
x = x.cuda(0)
print(x)
# 通过tensor的device属性来查看该 Tensor所在的设备
print(x.device)

tensor([1, 2, 3], device='cuda:0')
cuda:0


In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

x = torch.tensor([1, 2, 3], device=device)
print(x)
# 下面是另一种方式
x = torch.tensor([1, 2, 3]).to(device)
print(x)

tensor([1, 2, 3], device='cuda:0')
tensor([1, 2, 3], device='cuda:0')


In [18]:
# 如果对在 GPU 上的数据进行运算，那么结果还是存放在 GPu上的
y = x**2
print(y)

tensor([1, 4, 9], device='cuda:0')


In [0]:
# 需要注意的是，存储在不同位置中的数据是不可以直接进行运算的。及存放在cpu上的数据不可以直接与存放在GPU上的数据进行运算，位于不同GPU上的数据也是不能直接进行运算的
z = y + x.cpu()  # 这里就会报错了

In [21]:
# 4.6.3 模型的GPU计算
# 同 Tensor类似，Pytorch模型也可以通过 .cuda 转换到GPU上。我们可以通过检查模型的参数的device属性来查看存放模型的设备
net = nn.Linear(3, 1)
list(net.parameters())[0].device

device(type='cpu')

In [22]:
net = net.cuda()
list(net.parameters())[0].device

device(type='cuda', index=0)

In [23]:
# 同样的我们要保证 参与到模型运算的tensor都要在 GPU显存上，
x = torch.rand(2,3).cuda()
net(x)

tensor([[-0.1951],
        [-0.0600]], device='cuda:0', grad_fn=<AddmmBackward>)